# LLM Server для RFSD Agent (Colab)

Этот notebook запускает:
1. **Mistral-7B-Instruct** модель (бесплатно)
2. FastAPI сервер для обработки запросов
3. ngrok туннель для доступа извне

**⚠️ ВАЖНО: Настройте токен ngrok ПЕРЕД запуском:**

### Шаг 1: Получите токен ngrok
1. Откройте [ngrok.com/signup](https://dashboard.ngrok.com/signup) (бесплатно, 1 минута)
2. Зарегистрируйтесь через Google/GitHub или email
3. Скопируйте токен со страницы [Your Authtoken](https://dashboard.ngrok.com/get-started/your-authtoken)

### Шаг 2: Добавьте токен в Colab Secrets
1. Слева нажмите на иконку **🔑 Secrets**
2. Нажмите **+ Add new secret**
3. **Name**: `NGROK_TOKEN`
4. **Value**: вставьте ваш токен
5. Включите галочку **Notebook access**

### Шаг 3: Запустите notebook
1. Включите GPU: `Runtime` → `Change runtime type` → `T4 GPU`
2. Запустите все ячейки: `Runtime` → `Run all`

In [ ]:
# Шаг 1: Установка библиотек
!pip install -q transformers accelerate bitsandbytes fastapi uvicorn pyngrok

In [ ]:
# Шаг 2: Настройка ngrok токена (читаем из Colab Secrets)
from google.colab import userdata
from pyngrok import ngrok

try:
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
    ngrok.set_auth_token(NGROK_TOKEN)
    print("✅ Токен ngrok настроен (из Secrets)")
except Exception as e:
    print("❌ ОШИБКА: Токен ngrok не найден в Secrets!")
    print("\n📝 Как добавить токен в Secrets:")
    print("1. Слева нажмите на иконку 🔑 (Secrets)")
    print("2. Нажмите '+ Add new secret'")
    print("3. Name: NGROK_TOKEN")
    print("4. Value: ваш токен (получите на https://dashboard.ngrok.com/get-started/your-authtoken)")
    print("5. Включите галочку 'Notebook access'")
    print("6. Запустите эту ячейку снова")
    raise ValueError(f"Токен ngrok не настроен: {e}")

In [ ]:
# Шаг 3: Загрузка модели (Mistral-7B-Instruct)
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Используем 8-bit квантизацию для экономии памяти
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

print("🔄 Загрузка модели Mistral-7B...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)
print("✅ Модель загружена")

In [ ]:
# Шаг 4: FastAPI сервер
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
from threading import Thread

app = FastAPI(title="LLM Server")

class ChatRequest(BaseModel):
    messages: list[dict]
    temperature: float = 0.3
    max_tokens: int = 500

class ChatResponse(BaseModel):
    choices: list[dict]

@app.post("/v1/chat/completions")
async def chat_completions(request: ChatRequest) -> ChatResponse:
    """OpenAI-compatible endpoint."""
    # Формируем промпт из messages
    prompt_parts = []
    for msg in request.messages:
        role = msg['role']
        content = msg['content']
        if role == 'system':
            prompt_parts.append(f"System: {content}")
        elif role == 'user':
            prompt_parts.append(f"User: {content}")
    prompt_parts.append("Assistant:")
    prompt = "\n".join(prompt_parts)
    
    # Генерация
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=request.max_tokens,
        temperature=request.temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Извлекаем только ответ модели
    answer = generated_text.split("Assistant:")[-1].strip()
    
    return ChatResponse(
        choices=[
            {
                "message": {
                    "role": "assistant",
                    "content": answer
                }
            }
        ]
    )

@app.get("/health")
async def health():
    return {"status": "ok", "model": model_id}

# Запуск в фоновом потоке
def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

server_thread = Thread(target=run_server, daemon=True)
server_thread.start()

print("✅ FastAPI запущен на порту 8000")

In [ ]:
# Шаг 5: ngrok туннель
from pyngrok import ngrok
import time

# Даём серверу время запуститься
time.sleep(3)

# Создаём туннель
tunnel = ngrok.connect(8000)
public_url = tunnel.public_url  # Извлекаем URL как строку

print("="*60)
print("✅ LLM Server запущен!")
print("="*60)
print(f"📡 Публичный URL: {public_url}")
print("="*60)
print("\n📋 СКОПИРУЙТЕ ЭТОТ URL и добавьте в .env вашего бэкенда:")
print(f"\nLLM_ENDPOINT_URL={public_url}")
print("\nЗатем перезапустите бэкенд (Ctrl+C и uvicorn app.main:app --reload)")
print("="*60)

In [ ]:
# Шаг 6: Тестовый запрос (проверка работоспособности)
import requests

test_payload = {
    "messages": [
        {"role": "system", "content": "Ты финансовый аналитик."},
        {"role": "user", "content": "Объясни кратко: что такое выручка?"}
    ],
    "temperature": 0.3,
    "max_tokens": 100
}

print("🧪 Тестирую модель...\n")
response = requests.post(f"{public_url}/v1/chat/completions", json=test_payload, timeout=60)
print(f"✅ Статус: {response.status_code}")
print(f"\n💬 Ответ модели:\n{response.json()['choices'][0]['message']['content']}")

## ✅ Готово!

### Следующие шаги:

1. **Скопируйте URL** из ячейки выше (например: `https://abc123.ngrok-free.app`)

2. **Добавьте в бэкенд** (`services/rfsd_backend/.env`):
   ```env
   LLM_ENDPOINT_URL=https://abc123.ngrok-free.app
   ```

3. **Перезапустите бэкенд**:
   - В терминале: `Ctrl+C`
   - Затем: `uvicorn app.main:app --reload --port 8000`

4. **Тест в Telegram боте**:
   ```
   Проанализируй финансовое состояние ИНН 7722514880 за 2023 и дай выводы
   ```

---

### ⚠️ Важные заметки:

- **Время жизни:** Colab сессия живёт ~12 часов (без активности). Если отключилась — перезапустите `Runtime` → `Run all`.
- **URL изменится** при каждом перезапуске — обновляйте `.env`.
- **Первый запрос медленный** (~20-30 сек), затем быстрее (~5-10 сек).
- **ngrok бесплатно:** 1 активный туннель, неограниченное время.